## Phi-4 RAG with Azure AI Search

ఈ నోట్బుక్ Azure AI Search తో Retrieval Augmented Generation (RAG) కోసం Phi-4-mini మరియు Phi-4-multimodal ను 어떻게 ఉపయోగించాలో చూపిస్తుంది. ఇది సింగిల్-మోడాలిటీ (కేవలం టెక్స్ట్) మరియు మల్టీ-మోడాలిటీ (టెక్స్ట్ మరియు ఇమేజ్) రెండింటినీ పరామర్శిస్తుంది.

**ముందస్తు అవసరాలు:**
*   Azure AI Search vector index (ఒకటి సృష్టించడానికి [ఈ సూచనలు](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) అనుసరించండి)
*   Phi-4-mini లేదా Phi-4-multimodal ఎండ్‌పాయింట్లు Azure AI Foundryలో డిప్లాయ్ చేయబడినవి


In [ ]:
! pip install azure-ai-inference azure-search-documents

### పాఠ్య-మాత్ర RAG తో Phi-4-mini

ఈ విభాగం Phi-4-mini ను RAG కోసం చాట్ పూర్తి చేయడానికి మోడల్‌గా ఉపయోగించడం, ఇన్పుట్‌గా కేవలం పాఠ్యాన్ని ఉపయోగించడం ఎలా జరుగుతుందో చూపిస్తుంది.  ఇది Azure AI Foundry Inference మరియు Azure AI Search కు కనెక్ట్ కావడం, సంబంధిత డాక్యుమెంట్లను పొందడం, మరియు పొందిన సందర్భాన్ని ఉపయోగించి సమాధానాన్ని రూపొందించడం అనేది కలిగి ఉంటుంది.


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### Phi-4-multimodal తో బహుమోడల్ RAG

ఈ విభాగం RAG కోసం Phi-4-multimodal ను చాట్ కంప్లీషన్ మోడల్‌గా ఉపయోగించి, పాఠ్య మరియు చిత్రం ఇన్‌పుట్‌లను ఉమ్మడిగా తీసుకునే విధానాన్ని చూపిస్తుంది.  దీనిలో Azure AI Inference మరియు Azure AI Search‌కు కనెక్ట్ చేయడం, సంబంధిత డాక్యుమెంట్లను తిరిగి పొందడం, మరియు బహుమోడల్ ప్రతిస్పందనను రూపొందించడం ఎలా అన్నది ఉంటుంది.

**గమనిక:** మీకు Azure AI Search indexలో `text_vector` మరియు `image_vector` ఫీల్డ్‌లు రెండూ ఉంటే, మీరు మల్టీ-వెక్టర్ క్వెరీ కూడా నిర్వహించవచ్చు.


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
అస్పష్టం:
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator] (https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో తప్పులు లేదా అసత్యతలు ఉండవచ్చని దయచేసి గమనించండి. దీనిలోని స్థానిక భాషలోని అసలు పత్రాన్ని అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫార్సు చేసుకుంటాము. ఈ అనువాదం ఉపయోగం నుంచి ఉద్భవించే ఏదైనా అపార్థాలు లేదా తప్పుగా వివరణలకు మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
